In [3]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

C:\Users\thoma\AppData\Local\Temp\ipykernel_15264\2887495799.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [37]:
municipios = pd.read_excel("Dados/Municipios/lista-1262municipios-semiarido-2017.xlsx")
municipios = municipios.rename(columns=municipios.iloc[1])
municipios = municipios.drop([0, 1]).reset_index(drop=True)
municipios = municipios.drop('#', axis=1).reset_index(drop=True)
municipios_profile = ProfileReport(municipios, title="municipios")

In [11]:
idh_tot = pd.read_excel("Dados/Registros Administrativos/dados_registros_administrativos_total_2012_2017.xlsx")
idh_tot_profile = ProfileReport(idh_tot, title="idh_tot")

In [38]:
municipios

,Cod IBGE,UF,Município,População Estimada - 2017
0,2700102,AL,ÁGUA BRANCA,20467
1,2700300,AL,ARAPIRACA,234185
2,2700706,AL,BATALHA,18757
3,2700904,AL,BELO MONTE,6797
4,2701209,AL,CACIMBINHAS,10897
...,...,...,...,...
1257,2806008,SE,RIBEIRÓPOLIS,18641
1258,2807006,SE,SÃO MIGUEL DO ALEIXO,3943
1259,2807105,SE,SIMÃO DIAS,40838
1260,2807303,SE,TELHA,3221
